# Function profiling
* When is function level profiling appropriate?
* How can `cProfile` and `snakeviz` tools be used to profile a Python program?
* How are the outputs from function level profiling interpreted?

### Software is usually made up of a hierarchy of function calls:
* Some written by the developer

* Others from the standard library or third-party packages

#### `Function-level profiling` shows where time is being spent in functions.
* It counts how many times each function is called
  
* It measures total time spent in each function, including and excluding child function calls
This helps quickly find functions that take up a large share of the runtime.

#### In this episode, we will:

* Use the function-level profiler `cProfile`

* Visualise the output with `snakeviz`

* Learn how to interpret the results

In [7]:
# ![title](stack.png)

# What is a Call Stack?
* A Last-in-first-out queue
* It keeps track of function calls and their associated variables
* Info is kept as long as the functions are active, i.e., still running

Example of a function call stack using traceback package

In [ ]:
import traceback

def a():
    b1()
    b2()
    
def b1():
    pass
    
def b2():
    c()
    
def c():
    traceback.print_stack()

a()

#  File "/var/folders/s0/2zsybdkd50s2xdxzyg37j4611m1b1l/T/ipykernel_16681/3225044458.py", line 16, in <module>
#    a()
#  File "/var/folders/s0/2zsybdkd50s2xdxzyg37j4611m1b1l/T/ipykernel_16681/3225044458.py", line 5, in a
#    b2()
#  File "/var/folders/s0/2zsybdkd50s2xdxzyg37j4611m1b1l/T/ipykernel_16681/3225044458.py", line 11, in b2
#    c()
#  File "/var/folders/s0/2zsybdkd50s2xdxzyg37j4611m1b1l/T/ipykernel_16681/3225044458.py", line 14, in c
#    traceback.print_stack()

The stack trace shows the filename and line number of each function call.

You may see stack traces like this when an unhandled exception is thrown by your code.
Stack traces are very helpful to identify the source line of the error

# `cProfile`

`cProfile` is part of python standard library, i.e., no need to install it.
* you can import it from within your python code
* or run it in command line as follows:
  ``` bash
  python -m cProfile -o <output file.prof> <script name> <arguments>
  ```

Profiling outputs will be writtin to a file with `.prof` extension if you specify `-o  my_out.prof` argument
Otherwise, it will be written directly to your console output

In [5]:
# save it to func_prof.py
import time

def a_1():
    for i in range(3):
        b_1()
    time.sleep(1)
    b_2()
    
def b_1():
    c_1()
    c_2()

def b_2():
    time.sleep(1)
    
def c_1():
    time.sleep(0.5)

def c_2():
    time.sleep(0.3)
    d_1()

def d_1():
    time.sleep(0.1)

# Entry Point
a_1()

# a_1() calls 3 times b_1(), then sleeps for (1s), then calls b_2 (sleeping for 1s) => 2s
# b_1() calls c_1() --> c_1() is called 3 times (sleeping in total 1.5s) => 1.5s
# b_1() calls c_2() --> c_2() is called 3 times (sleeping in total 0.9s) then sleeping 0.1*3=0.3 => total=1.2s
# .sleep() is called: 11 times (1 in a_1(), 3 times in c_1(), 3 times in c_2() and 3 times in d_1() and Once in b_2())
# Total time spend: 4.7s

# Run `cProfile` module in command line. 
We won't specify the output file for the moment
```bash
python -m cProfile func_prof.py
```


```output
         28 function calls in 4.736 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.736    4.736 {built-in method builtins.exec}
        1    0.000    0.000    4.736    4.736 func_prof2.py:1(<module>)
        1    0.000    0.000    4.736    4.736 func_prof2.py:3(a_1)
       11    4.736    0.431    4.736    0.431 {built-in method time.sleep}
        3    0.000    0.000    2.732    0.911 func_prof2.py:9(b_1)
        3    0.000    0.000    1.509    0.503 func_prof2.py:16(c_1)
        3    0.000    0.000    1.223    0.408 func_prof2.py:19(c_2)
        1    0.000    0.000    1.004    1.004 func_prof2.py:13(b_2)
        3    0.000    0.000    0.311    0.104 func_prof2.py:23(d_1)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
```

* `ncalls`: number of times (in seconds) a function was called
* `tottime`: total time spent in that function, excluding child function
* `percall`: average time spent in that function each time, i.e., `tottime/ncalls`
* `cumtime`: total time spent in that function, including child function calls
* `percall`: The average `cumtime` per function call (`cumtime/ncalls`).
* `filename:lineno(function)`: function name and location 

# Let's visualise the profiling results using `snakeviz`
We should run cProfile and specify an output file:
```bash
python -m cProfile -o out.prof func_prof.py
```

# Install snakeviz if not already done in the setup

```bash
pip install snakeviz
```

* verify it is installed by running `snakeviz --version`

# Run snakeviz in your command line:
```bash
python -m snakeviz out.prof
```
In consequence this will open out.prof in your browser

# Run snakeviz in jupyter notebook:

In [3]:
!pip install snakeviz
%load_ext snakeviz

In [2]:
# run snakeviz in jupyter notebook
# %snakeviz a_1()

# EXERCISE 1: TRAVELLING SALESPERSON
Download the python code and save it in an accessible folder
```bash
python -m cProfile travellingsales.py <integer, nb of cities>
```

TODO: 
analyse the code and provide a summary, explaining the key points of the code.
Running the profiling multiple times with different number of cities to showcase the importance of inputs in catching the bottlenecks, and also showing the tradeoff between choosing the fastest setup to catch the bottlenecks

In [8]:
# save it as travellingsales.py
"""
Naive brute force travelling salesperson
python travellingsales.py <cities>
"""

import itertools
import math
import random
import sys

# calculates euclidean distance between two points
def distance(point1, point2):
    return math.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)

# calculates total distance among a set of points in a specific order
def total_distance(points, order):
    total = 0
    for i in range(len(order) - 1):
        total += distance(points[order[i]], points[order[i + 1]])
    return total + distance(points[order[-1]], points[order[0]])

# calculates the shortest path to link as set of points
def traveling_salesman_brute_force(points):
    min_distance = float('inf')
    min_path = None
    # create permutations: factorial n!
    for order in itertools.permutations(range(len(points))):
        d = total_distance(points, order)
        if d < min_distance:
            min_distance = d
            min_path = order
    return min_path, min_distance

# Argument parsing
if len(sys.argv) != 2:
    print("Script expects 1 positive integer argument, %u found."%(len(sys.argv) - 1))
    sys.exit()
cities_len = int(sys.argv[1])
if cities_len < 1:
    print("Script expects 1 positive integer argument, %s converts < 1."%(sys.argv[1]))
    sys.exit()
# Define the cities as (x, y) coordinates
random.seed(12) # Fixed random for consistency
cities = [(0,0)]
for i in range(cities_len):
    cities.append((random.uniform(-1, 1), random.uniform(-1, 1)))

# Find the shortest path
shortest_path, min_distance = traveling_salesman_brute_force(cities)
print("Cities:", cities_len)
print("Shortest Path:", shortest_path)
print("Shortest Distance:", min_distance)

Script expects 1 positive integer argument, 2 found.


SystemExit: 

In [4]:
# to calculate the shortest distance between a set of points,
# first, we need to find the number of possible paths: factorial of the number of points (n!)
# Second, calculate the total path for each possible path
# Finally, find the minimum path and return the path and the minimal distance

# for, 5 points, there are 5*4*3*2*1 = math.factorial(5) = 120 possibilities
# for 10 points, there are 10*9*8*..*5*4*3*2*1 = math.factorial(10) = 3 628 800 possibilities!!!
import math
math.factorial(10)

3628800

In [ ]:
# 39 916 800   77.465    0.000  201.843    0.000 travellingsales.py:14(total_distance)
# 439 084 800  109.961    0.000  123.194    0.000 travellingsales.py:11(distance)

### run the cProfile tool on this file
```bash
python -m cprofile -p out2.prof travellingsales.py 1
python -m cprofile -p out2.prof travellingsales.py 5
python -m cprofile -p out2.prof travellingsales.py 10

# visualise
python -m snakeviz out2.prof 
```

The hotspot only becomes visible when an argument of 5 or greater is passed.

You should see that distance() (from `travellingsales.py:11`) becomes the largest box (similarly it’s parent in the call-stack `total_distance()`) showing that it scales poorly with the number of cities. 
With 5 cities, `distance()` has a cumulative time of `~35%` the runtime, this increases to `~60% with 9 cities`.

Other boxes within the diagram correspond to the initialisation of imports, or initialisation of cities. These have constant or linear scaling, so their cost barely increases with the number of cities.

This highlights the need to profile a realistic test-case expensive enough that initialisation costs are not the most expensive component.

# 15min BREAK NOW